In [ ]:
!pip install catboost
!pip install xgboost
!pip install lime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=ce732cbfe3cc2fede785c23222540d517d0920de99b53456b821e028050da6b6
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
!pip install shap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 6.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import plotly.express as px
import plotly.offline as pyo
import seaborn as sns

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

# !pip install interpret
# from interpret.blackbox import LimeTabular
# from interpret import show

import lime
import lime.lime_tabular
from lime.lime_tabular import LimeTabularExplainer
import graphviz
import shap

import pickle

import warnings
warnings.filterwarnings("ignore")

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/ids-intrusion-csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import os
os.chdir("drive/My Drive/zeynep")
!ls

data


In [ ]:
%%time
network_data_d1 = pd.read_csv("data/02-14-2018.csv", low_memory=False)
network_data_d2 = pd.read_csv("data/02-15-2018.csv", low_memory=False)
network_data_d3 = pd.read_csv("data/02-16-2018.csv", low_memory=False)
network_data_d4 = pd.read_csv("data/02-20-2018.csv", low_memory=False)


CPU times: user 55.2 s, sys: 6.15 s, total: 1min 1s
Wall time: 1min 11s


In [ ]:
network_data_d4.drop(columns=['Flow ID', 'Src IP', 'Src Port', 'Dst IP'], axis=1,inplace=True)

KeyError: "['Flow ID', 'Src IP', 'Src Port', 'Dst IP'] not found in axis"

In [ ]:
def fixDataType(df_dataset):

    df_dataset = df_dataset[df_dataset['Dst Port'] != 'Dst Port']

    df_dataset['Dst Port'] = df_dataset['Dst Port'].astype(int)
    df_dataset['Protocol'] = df_dataset['Protocol'].astype(int)
    df_dataset['Flow Duration'] = df_dataset['Flow Duration'].astype(int)
    df_dataset['Tot Fwd Pkts'] = df_dataset['Tot Fwd Pkts'].astype(int)
    df_dataset['Tot Bwd Pkts'] = df_dataset['Tot Bwd Pkts'].astype(int)
    df_dataset['TotLen Fwd Pkts'] = df_dataset['TotLen Fwd Pkts'].astype(int)
    df_dataset['TotLen Bwd Pkts'] = df_dataset['TotLen Bwd Pkts'].astype(int)
    df_dataset['Fwd Pkt Len Max'] = df_dataset['Fwd Pkt Len Max'].astype(int)
    df_dataset['Fwd Pkt Len Min'] = df_dataset['Fwd Pkt Len Min'].astype(int)
    df_dataset['Fwd Pkt Len Mean'] = df_dataset['Fwd Pkt Len Mean'].astype(float)
    df_dataset['Fwd Pkt Len Std'] = df_dataset['Fwd Pkt Len Std'].astype(float)
    df_dataset['Bwd Pkt Len Max'] = df_dataset['Bwd Pkt Len Max'].astype(int)
    df_dataset['Bwd Pkt Len Min'] = df_dataset['Bwd Pkt Len Min'].astype(int)
    df_dataset['Bwd Pkt Len Mean'] = df_dataset['Bwd Pkt Len Mean'].astype(float)
    df_dataset['Bwd Pkt Len Std'] = df_dataset['Bwd Pkt Len Std'].astype(float)
    df_dataset['Flow Byts/s'] = df_dataset['Flow Byts/s'].astype(float)
    df_dataset['Flow Pkts/s'] = df_dataset['Flow Pkts/s'].astype(float)
    df_dataset['Flow IAT Mean'] = df_dataset['Flow IAT Mean'].astype(float)
    df_dataset['Flow IAT Std'] = df_dataset['Flow IAT Std'].astype(float)
    df_dataset['Flow IAT Max'] = df_dataset['Flow IAT Max'].astype(int)
    df_dataset['Flow IAT Min'] = df_dataset['Flow IAT Min'].astype(int)
    df_dataset['Fwd IAT Tot'] = df_dataset['Fwd IAT Tot'].astype(int)
    df_dataset['Fwd IAT Mean'] = df_dataset['Fwd IAT Mean'].astype(float)
    df_dataset['Fwd IAT Std'] = df_dataset['Fwd IAT Std'].astype(float)
    df_dataset['Fwd IAT Max'] = df_dataset['Fwd IAT Max'].astype(int)
    df_dataset['Fwd IAT Min'] = df_dataset['Fwd IAT Min'].astype(int)
    df_dataset['Bwd IAT Tot'] = df_dataset['Bwd IAT Tot'].astype(int)
    df_dataset['Bwd IAT Mean'] = df_dataset['Bwd IAT Mean'].astype(float)
    df_dataset['Bwd IAT Std'] = df_dataset['Bwd IAT Std'].astype(float)
    df_dataset['Bwd IAT Max'] = df_dataset['Bwd IAT Max'].astype(int)
    df_dataset['Bwd IAT Min'] = df_dataset['Bwd IAT Min'].astype(int)
    df_dataset['Fwd PSH Flags'] = df_dataset['Fwd PSH Flags'].astype(int)
    df_dataset['Bwd PSH Flags'] = df_dataset['Bwd PSH Flags'].astype(int)
    df_dataset['Fwd URG Flags'] = df_dataset['Fwd URG Flags'].astype(int)
    df_dataset['Bwd URG Flags'] = df_dataset['Bwd URG Flags'].astype(int)
    df_dataset['Fwd Header Len'] = df_dataset['Fwd Header Len'].astype(int)
    df_dataset['Bwd Header Len'] = df_dataset['Bwd Header Len'].astype(int)
    df_dataset['Fwd Pkts/s'] = df_dataset['Fwd Pkts/s'].astype(float)
    df_dataset['Bwd Pkts/s'] = df_dataset['Bwd Pkts/s'].astype(float)
    df_dataset['Pkt Len Min'] = df_dataset['Pkt Len Min'].astype(int)
    df_dataset['Pkt Len Max'] = df_dataset['Pkt Len Max'].astype(int)
    df_dataset['Pkt Len Mean'] = df_dataset['Pkt Len Mean'].astype(float)
    df_dataset['Pkt Len Std'] = df_dataset['Pkt Len Std'].astype(float)
    df_dataset['Pkt Len Var'] = df_dataset['Pkt Len Var'].astype(float)
    df_dataset['FIN Flag Cnt'] = df_dataset['FIN Flag Cnt'].astype(int)
    df_dataset['SYN Flag Cnt'] = df_dataset['SYN Flag Cnt'].astype(int)
    df_dataset['RST Flag Cnt'] = df_dataset['RST Flag Cnt'].astype(int)
    df_dataset['PSH Flag Cnt'] = df_dataset['PSH Flag Cnt'].astype(int)
    df_dataset['ACK Flag Cnt'] = df_dataset['ACK Flag Cnt'].astype(int)
    df_dataset['URG Flag Cnt'] = df_dataset['URG Flag Cnt'].astype(int)
    df_dataset['CWE Flag Count'] = df_dataset['CWE Flag Count'].astype(int)
    df_dataset['ECE Flag Cnt'] = df_dataset['ECE Flag Cnt'].astype(int)
    df_dataset['Down/Up Ratio'] = df_dataset['Down/Up Ratio'].astype(int)
    df_dataset['Pkt Size Avg'] = df_dataset['Pkt Size Avg'].astype(float)
    df_dataset['Fwd Seg Size Avg'] = df_dataset['Fwd Seg Size Avg'].astype(float)
    df_dataset['Bwd Seg Size Avg'] = df_dataset['Bwd Seg Size Avg'].astype(float)
    df_dataset['Fwd Byts/b Avg'] = df_dataset['Fwd Byts/b Avg'].astype(int)
    df_dataset['Fwd Pkts/b Avg'] = df_dataset['Fwd Pkts/b Avg'].astype(int)
    df_dataset['Fwd Blk Rate Avg'] = df_dataset['Fwd Blk Rate Avg'].astype(int)
    df_dataset['Bwd Byts/b Avg'] = df_dataset['Bwd Byts/b Avg'].astype(int)
    df_dataset['Bwd Pkts/b Avg'] = df_dataset['Bwd Pkts/b Avg'].astype(int)
    df_dataset['Bwd Blk Rate Avg'] = df_dataset['Bwd Blk Rate Avg'].astype(int)
    df_dataset['Subflow Fwd Pkts'] = df_dataset['Subflow Fwd Pkts'].astype(int)
    df_dataset['Subflow Fwd Byts'] = df_dataset['Subflow Fwd Byts'].astype(int)
    df_dataset['Subflow Bwd Pkts'] = df_dataset['Subflow Bwd Pkts'].astype(int)
    df_dataset['Subflow Bwd Byts'] = df_dataset['Subflow Bwd Byts'].astype(int)
    df_dataset['Init Fwd Win Byts'] = df_dataset['Init Fwd Win Byts'].astype(int)
    df_dataset['Init Bwd Win Byts'] = df_dataset['Init Bwd Win Byts'].astype(int)
    df_dataset['Fwd Act Data Pkts'] = df_dataset['Fwd Act Data Pkts'].astype(int)
    df_dataset['Fwd Seg Size Min'] = df_dataset['Fwd Seg Size Min'].astype(int)
    df_dataset['Active Mean'] = df_dataset['Active Mean'].astype(float)
    df_dataset['Active Std'] = df_dataset['Active Std'].astype(float)
    df_dataset['Active Max'] = df_dataset['Active Max'].astype(int)
    df_dataset['Active Min'] = df_dataset['Active Min'].astype(int)
    df_dataset['Idle Mean'] = df_dataset['Idle Mean'].astype(float)
    df_dataset['Idle Std'] = df_dataset['Idle Std'].astype(float)
    df_dataset['Idle Max'] = df_dataset['Idle Max'].astype(int)
    df_dataset['Idle Min'] = df_dataset['Idle Min'].astype(int)

    return df_dataset

In [ ]:
%%time
# network_data_d1 = fixDataType(network_data_d1)
# network_data_d2 = fixDataType(network_data_d2)
network_data_d3 = fixDataType(network_data_d3)
# network_data_d4 = fixDataType(network_data_d4)


In [ ]:
# network_data_d1.shape
# print ('Number of rows (Samples): ' , network_data_d1.shape[0])
# print ('Number of columns (Features): ' , network_data_d1.shape[1])

In [ ]:
# network_data_d1

In [ ]:
# network_data_d1.info()

In [ ]:
# network_data_d1['Label'].value_counts()

In [ ]:
def dataProperties(df, day):
    print(day)
    df.shape
    print ('Number of rows (Samples): ' , df.shape[0])
    print ('Number of columns (Features): ' , df.shape[1])
#     print(df)
#     print(df.info())
    print(df['Label'].value_counts())
    print("\n\n\n")

In [ ]:
%%time
dataProperties(network_data_d1, "Day 1")
dataProperties(network_data_d2, "Day 2")
dataProperties(network_data_d3, "Day 3")
dataProperties(network_data_d4, "Day 4")


In [ ]:
# network_data['Label'].value_counts()

In [ ]:
# %%time
# # bar chart of packets label
# plt.figure(figsize=(30, 5))
# plt.title('Packet Distribution')
# # plt.bar(x=['Benign', 'FTP-BruteForce', 'SSH-Bruteforce'], height=network_data['Label'].value_counts(), color=['blue', 'magenta', 'cyan'])
# plt.bar(x=network_data['Label'].unique(), height=network_data['Label'].value_counts())
# p = plt.gcf()

In [ ]:
# %%time
def visualizeBar(df):
    # bar chart of packets label
    plt.figure(figsize=(10, 5))
    plt.title('Packet Distribution')
    # plt.bar(x=['Benign', 'FTP-BruteForce', 'SSH-Bruteforce'], height=network_data['Label'].value_counts(), color=['blue', 'magenta', 'cyan'])
    plt.bar(x=df['Label'].unique(), height=df['Label'].value_counts())
    p = plt.gcf()

In [ ]:
%%time
# network_data = dropInfinateNull(network_data)
visualizeBar(network_data_d1)
visualizeBar(network_data_d2)
visualizeBar(network_data_d3)
visualizeBar(network_data_d4)


In [ ]:
# %%time
# # pie chart of packets label
# plt.figure(figsize=(5, 5))
# circle = plt.Circle((0, 0), 0.7, color='white')
# plt.title('Packet Distribution')
# # plt.pie(network_data_d1['Label'].value_counts(), labels=['Benign', 'FTP-BruteForce', 'SSH-Bruteforce'], colors=['blue', 'magenta', 'cyan'])
# plt.pie(network_data['Label'].value_counts(), labels=network_data['Label'].unique())
# p = plt.gcf()
# p.gca().add_artist(circle)

In [ ]:
# %%time
def visualizePie(df):
    # pie chart of packets label
    plt.figure(figsize=(5, 5))
    circle = plt.Circle((0, 0), 0.7, color='white')
    plt.title('Packet Distribution')
    # plt.pie(network_data_d1['Label'].value_counts(), labels=['Benign', 'FTP-BruteForce', 'SSH-Bruteforce'], colors=['blue', 'magenta', 'cyan'])
    plt.pie(df['Label'].value_counts(), labels=df['Label'].unique())
    p = plt.gcf()
    p.gca().add_artist(circle)

In [ ]:
%%time
# network_data = dropInfinateNull(network_data)
visualizePie(network_data_d1)
visualizePie(network_data_d2)
visualizePie(network_data_d3)
visualizePie(network_data_d4)


In [ ]:
# print (network_data_d1.shape)

# # replace infinity value as null value
# network_data_d1 = network_data_d1.replace(["Infinity", "infinity"], np.inf)
# network_data_d1 = network_data_d1.replace([np.inf, -np.inf], np.nan)

# # drop all null values
# network_data_d1.dropna(inplace=True)

# print (network_data_d1.shape)

In [ ]:
def dropInfinateNull(df):
    print (df.shape)

    # replace infinity value as null value
    df = df.replace(["Infinity", "infinity"], np.inf)
    df = df.replace([np.inf, -np.inf], np.nan)

    # drop all null values
    df.dropna(inplace=True)

    print (df.shape)

    return df

In [ ]:
%%time
# network_data = dropInfinateNull(network_data)
network_data_d1 = dropInfinateNull(network_data_d1)
network_data_d2 = dropInfinateNull(network_data_d2)
network_data_d3 = dropInfinateNull(network_data_d3)
network_data_d4 = dropInfinateNull(network_data_d4)


In [ ]:
# %%time
# network_data.drop(columns="Timestamp", inplace=True)
# print (network_data.shape)

In [ ]:
def dropUnnecessaryColumn(df):
    df.drop(columns="Timestamp", inplace=True)
    print (df.shape)
    return df

In [ ]:
%%time
# network_data = dropInfinateNull(network_data)
network_data_d1 = dropUnnecessaryColumn(network_data_d1)
network_data_d2 = dropUnnecessaryColumn(network_data_d2)
network_data_d3 = dropUnnecessaryColumn(network_data_d3)
network_data_d4 = dropUnnecessaryColumn(network_data_d4)


In [ ]:
# %%time
# # encode the target feature
# network_data['Label'] = network_data['Label'].apply(lambda x: "Benign" if x == 'Benign' else "Malicious")
# print(network_data['Label'].unique())

In [ ]:
# network_data['Label'].value_counts()

In [ ]:
%%time
def transformTargetLabelToBinary(df):
    # encode the target feature
    df['Label'] = df['Label'].apply(lambda x: "Benign" if x == 'Benign' else "Malicious")
    print(df['Label'].unique())
    print(df['Label'].value_counts())
    return df

In [ ]:
%%time
# network_data = dropInfinateNull(network_data)
network_data_d1 = transformTargetLabelToBinary(network_data_d1)
network_data_d2 = transformTargetLabelToBinary(network_data_d2)
network_data_d3 = transformTargetLabelToBinary(network_data_d3)
network_data_d4 = transformTargetLabelToBinary(network_data_d4)


In [ ]:
# %%time
# # split data into features and target
# X=network_data.drop(["Label"], axis=1)
# y=network_data["Label"]

In [ ]:
%%time
# # applying oversampling
# ros = RandomOverSampler()
# X_balanced, y_balanced = ros.fit_resample(X, y) # insted of X, y use the direct syntex

In [ ]:
# # applying oversampling
# rus = RandomUnderSampler()
# X_balanced, y_balanced = rus.fit_resample(X, y) # insted of X, y use the direct syntex

In [ ]:
%%time
# # applying SMOTE
# smote = SMOTE(random_state=42)
# X_balanced, y_balanced = smote.fit_resample(X, y)

In [ ]:
# %%time
# network_data = pd.concat([X_balanced, y_balanced], axis=1)
# del X, y, X_balanced, y_balanced
# print (network_data.shape)
# print(network_data['Label'].value_counts())

In [ ]:
%%time
def balanceData(df):
    # split data into features and target
    X=df.drop(["Label"], axis=1)
    y=df["Label"]

    # applying oversampling
    rus = RandomUnderSampler()
    X_balanced, y_balanced = rus.fit_resample(X, y) # insted of X, y use the direct syntex

    df = pd.concat([X_balanced, y_balanced], axis=1)
    del X, y, X_balanced, y_balanced
    print (df.shape)
    print(df['Label'].value_counts())

    return df

In [ ]:
%%time
# network_data = dropInfinateNull(network_data)
network_data_d1 = balanceData(network_data_d1)
network_data_d2 = balanceData(network_data_d2)
network_data_d3 = balanceData(network_data_d3)
network_data_d4 = balanceData(network_data_d4)


In [ ]:
%%time
network_data = pd.concat([network_data_d1, network_data_d2], axis=0)
network_data.reset_index(drop=True, inplace=True)
del network_data_d1, network_data_d2

network_data = pd.concat([network_data, network_data_d3], axis=0)
network_data.reset_index(drop=True, inplace=True)
del network_data_d3

network_data = network_data_d3
del network_data_d3





In [ ]:
network_data['Label'].value_counts()

In [ ]:
%%time
# while working comment one line and uncomment another as needed
# temp_network_data = network_data
# network_data = temp_network_data

In [ ]:
%%time
# drop the constant columns (which varience is 0)
variances = network_data.var(numeric_only=True)
constant_columns = variances[variances == 0].index
network_data = network_data.drop(constant_columns, axis=1)

print(constant_columns)
print (network_data.shape)

In [ ]:
%%time
duplicates = set()
for i in range(0, len(network_data.columns)):
    col1 = network_data.columns[i]
    for j in range(i+1, len(network_data.columns)):
        col2 = network_data.columns[j]
        if(network_data[col1].equals(network_data[col2])):
            duplicates.add(col2)

print (duplicates)
network_data.drop(duplicates, axis=1, inplace=True)
print (network_data.shape)

In [ ]:
%%time
# encode the target feature
network_data['Label'] = network_data['Label'].apply(lambda x: 0 if x == 'Benign' else 1)
print(network_data['Label'].unique())

In [ ]:
%%time
# pearson correlation heatmap
plt.figure(figsize=(70, 70))
corr = network_data.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap='RdBu', vmin=-1, vmax=1, square=True) # annot=True
plt.show()

In [ ]:
%%time
correlated_col = set()
is_correlated = [True] * len(corr.columns)
threshold = 0.90
for i in range (len(corr.columns)):
    if(is_correlated[i]):
        for j in range(i):
          if (corr.iloc[i, j] >= threshold) and (is_correlated[j]):
            colname = corr.columns[j]
            is_correlated[j]=False
            correlated_col.add(colname)

print(correlated_col)
print(len(correlated_col))

In [ ]:
%%time
network_data.drop(correlated_col, axis=1, inplace=True)
print (network_data.shape)

In [ ]:
%%time
# pearson correlation heatmap
plt.figure(figsize=(70, 70))
corr = network_data.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap='RdBu', vmin=-1, vmax=1, square=True) # annot=True
plt.show()

In [ ]:
%%time
# while working comment one line and uncomment another as needed
# temp_network_data = network_data
# network_data = temp_network_data

In [ ]:
%%time
# while working comment one line and uncomment another as needed
# temp_network_data = network_data
# network_data = temp_network_data

In [ ]:
# # split data into features and target
# X=network_data_d1.drop(["Label"], axis=1)
# y=network_data_d1["Label"]

In [ ]:
# %%time
# # applying RFE with CV
# dt = DecisionTreeClassifier()
# cv = StratifiedKFold(5)
# min_features = 1

# rfecv = RFECV(estimator=dt, step=1, cv=cv, scoring="accuracy", min_features_to_select=min_features, n_jobs=2)
# rfecv.fit(X, y)

In [ ]:
# num_selected_features = rfecv.n_features_
# selected_features = X.columns[rfecv.support_]
# print(num_selected_features)
# print(selected_features)

In [ ]:
# network_data_d1 = network_data_d1.loc[:, selected_features]
# network_data_d1

In [ ]:
# # presenting number of feature vs accuracy
# num_scores = len(rfecv.cv_results_["mean_test_score"])
# plt.figure()
# plt.xlabel("Number of Selected Features")
# plt.ylabel("Mean Test Accuracy")
# plt.errorbar(
#     range(min_features, num_scores + min_features),
#     rfecv.cv_results_["mean_test_score"],
#     yerr=rfecv.cv_results_["std_test_score"],
# )
# plt.title("Recursive Feature Elimination")
# plt.show()

In [ ]:
%%time
# split data into features and target
X=network_data.drop(["Label"], axis=1)
y=network_data["Label"]

In [ ]:
# %%time
# # applying forward feature selection
# dt = DecisionTreeClassifier()
# sfs_forward = SequentialFeatureSelector(
#     dt, n_features_to_select=25, direction="forward"
# ).fit(X, y)

In [ ]:
# selected_features = X.columns[sfs_forward.get_support()]
# print(selected_features)

In [ ]:
%%time
# applying backward feature selection
dt = DecisionTreeClassifier()
sfs_backward = SequentialFeatureSelector(
    dt, n_features_to_select=30, direction="backward"
).fit(X, y)

In [ ]:
selected_features = X.columns[sfs_backward.get_support()]
print(selected_features)

In [ ]:
# add target label
selected_features = selected_features.tolist()
selected_features.append('Label')
selected_features = pd.Index(selected_features)
print(selected_features)

In [ ]:
%%time
network_data = network_data.loc[:, selected_features]
del X, y
network_data

In [ ]:
%%time
# while working comment one line and uncomment another as needed
# temp_network_data = network_data
# network_data = temp_network_data

In [ ]:
%%time
# list numeric columns
# numeric_cols = network_data.select_dtypes(include=[np.number]).columns.tolist()

numeric_cols = ['Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg',
       'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
       'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
       'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
       'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts',
       'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
       'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']

print(numeric_cols)

In [ ]:
%%time
# apply z-score normalization
# std = StandardScaler()
# network_data[numeric_cols] = std.fit_transform(network_data[numeric_cols])

In [ ]:
%%time
# apply min-max normalization
# mnmx = MinMaxScaler()
# network_data[numeric_cols] = mnmx.fit_transform(network_data[numeric_cols])

In [ ]:
%%time
# apply robust normalization
rbst = RobustScaler()
network_data[numeric_cols] = rbst.fit_transform(network_data[numeric_cols])

In [ ]:
network_data

In [ ]:
%%time
X=network_data.drop(["Label"], axis=1)
y=network_data["Label"]

# split the data for evaluation
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state =42, shuffle=True)

# K-fold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
def confusionMatrixHeatMap(cm, title):
    # box lebels
    group_counts = ["{0:0.0f}\n".format(value) for value in cm.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cm.flatten()/np.sum(cm)]
    box_labels = [f"{v1}{v2}".strip() for v1, v2 in zip(group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cm.shape[0],cm.shape[1])

    # categories
    categories = ['Benign', 'Malicious']

    # create a heatmap of the confusion matrix
    sns.heatmap(cm, annot=box_labels, fmt='', cmap='Blues', cbar=False, xticklabels=categories, yticklabels=categories)

    # create and add rectangle patch
    ax = plt.gca()
    rect = patches.Rectangle((0, 0), len(cm[0]), len(cm), linewidth=2, edgecolor='black', facecolor='none')
    ax.add_patch(rect)

    # set labels, title, and axis ticks
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    title_font = { 'fontsize': 16, 'fontname': 'Times New Roman' }
#     plt.title('Decision Tree\n', **title_font)
    plt.title(title + '\n', **title_font)

    # calculate accuracy and misclassification rate
    total_samples = len(y)
    correct_predictions = sum(y == y_pred)
    incorrect_predictions = total_samples - correct_predictions
    accuracy = correct_predictions / total_samples
    misclassification_rate = incorrect_predictions / total_samples

    # add accuracy and misclassification rate to the heatmap
    plt.text(0.5, -0.1, f'\n\n\nAccuracy: {accuracy:.2f}', ha='center', va='center', transform=plt.gca().transAxes)
    plt.text(0.5, -0.2, f'\nMisclassification Rate: {misclassification_rate:.4f}', ha='center', va='center', transform=plt.gca().transAxes)

    # Show the plot
    plt.show()

In [ ]:
# LazyClassifier
# clf = LazyClassifier(verbose = 0,
#                      ignore_warnings = True,
#                      custom_metric = None,
#                      predictions = False,
#                      random_state = 12,
#                      classifiers = 'all')

# models, predictions = clf.fit(X_train, X_test, y_train, y_test)
# models

In [ ]:
%%time
# create a Decision Tree model
dt = DecisionTreeClassifier()

# predict
y_pred = cross_val_predict(dt, X, y, cv=kf)


print ("Dicision Tree")
# generate report
cm=confusion_matrix(y, y_pred)
cr=classification_report(y, y_pred)
auc = roc_auc_score(y, y_pred)

print("Confusion Matrix:")
print(cm)

print("Performance Matrix:")
print(cr)

print("AUC:", auc)

confusionMatrixHeatMap(cm, title="Decision Tree")

In [ ]:
# %%time
# # create a Extra Trees model
# et = ExtraTreeClassifier()

# # predict
# y_pred = cross_val_predict(et, X, y, cv=kf)

# print ("Extra Tree")
# # generate report
# cm=confusion_matrix(y, y_pred)
# cr=classification_report(y, y_pred)

# print("Confusion Matrix:")
# print(cm)

# print("Performance Matrix:")
# print(cr)

In [ ]:
# %%time
# # create a Random Forest model
# rf = RandomForestClassifier()

# # predict
# y_pred = cross_val_predict(rf, X, y, cv=kf)

# print ("Random Forest")
# # generate report
# cm=confusion_matrix(y, y_pred)
# cr=classification_report(y, y_pred)

# print("Confusion Matrix:")
# print(cm)

# print("Performance Matrix:")
# print(cr)

In [ ]:
# %%time
# # create a Extra Trees model
# ett = ExtraTreesClassifier()

# # predict
# y_pred = cross_val_predict(ett, X, y, cv=kf)

# print ("Extra Trees")
# # generate report
# print("Confusion Matrix:")
# print(confusion_matrix(y, y_pred))

# print("Performance Matrix:")
# print(classification_report(y, y_pred))

In [ ]:
 %%time
# # create a KNN model
# knn = KNeighborsClassifier()

# # predict
# y_pred = cross_val_predict(knn, X, y, cv=kf)

# print ("KNN")
# # generate report
# print("Confusion Matrix:")
# print(confusion_matrix(y, y_pred))

# print("Performance Matrix:")
# print(classification_report(y, y_pred))

In [ ]:
X=network_data.drop(["Label"], axis=1)
y=network_data["Label"]

# split the data for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state =42, shuffle=True)

# K-fold
# kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
%%time
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
cr = classification_report(y_test, y_pred)
print(cr)

In [ ]:
%%time
# Save the model
with open('model.pkl', 'wb') as file:
    pickle.dump(dtc, file)

# # Load the model
# with open('model.pkl', 'rb') as file:
#     loaded_model = pickle.load(file)

In [ ]:
%%time
test_sample = X_test.iloc[0, :]
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X.columns.tolist(), verbose=True, mode='classification')
exp = explainer.explain_instance(test_sample.values, dtc.predict_proba)
exp.show_in_notebook()

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_test, y_pred)

In [ ]:
%%time
dot_data = tree.export_graphviz(dt, out_file=None, feature_names=X.columns, class_names=["0", "1"], filled=True)
graph = graphviz.Source(dot_data)
graph.render("induced_dt_tree_view")
graph

In [ ]:
explainer = shap.Explainer(dtc)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test, plot_type='bar')
plt.show()